In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba as nb

In [2]:
def e_integrand(zeta,m_e,T):
    integrand = np.zeros(len(zeta))
    for i in range(len(zeta)):
        integrand[i] = (zeta[i]**2*np.sqrt(zeta[i]**2+(m_e/T[i])**2))/(np.exp(np.sqrt(zeta[i]**2+(m_e/T[i])**2))+1)*np.exp(zeta[i])
    return integrand

x_gauss,w_gauss = np.polynomial.laguerre.laggauss(40)

In [3]:
def imp_values(filename,saved_file):
    actual_data= np.load(filename, allow_pickle=True)
    f_array = actual_data['fe']
    e_array = actual_data['e']
    temp_cm = actual_data['Tcm']
    m_s = actual_data['mass']#in MeV
    t = actual_data['time'] #inverse MeV
    a = actual_data['scalefactors']
    life = actual_data['lifetime']/(6.58*10**-25)*1/1000 #seconds need to be converted to inverse MeV
    temp = actual_data['temp'] #units of MeV
    decay = actual_data['decayrate']
    coll = actual_data['collisionrate']
    p_n = actual_data['p_n_rate']
    n_p = actual_data['n_p_rate']
    
    
    D = (10.75/61.75)
    Rz_3 = 1.202056901178332
    n_s = D*(3*Rz_3/(2*np.pi**2))*(temp_cm)**3*np.exp(-t/life)
    
    
    m_pl = (1.2*10**19)*1000 #planck mass in MeV
    m_e = 0.510 #electron mass in MeV
    
    
    p_y = (np.pi**2/15)*(temp)**4
    p_vs = m_s*n_s

    results = []
    for e,f in zip(e_array,f_array):
        result = np.trapz(e**3*f,e)
        results.append(result)
        
    p_v = 6*((temp_cm)**4/(2*np.pi**2))*results #in units of MeV^4
    
    integrand_vals = e_integrand(x_gauss,m_e,temp)
    vals = np.zeros(len(integrand_vals))
    
    for i in range(len(integrand_vals)):
        vals[i] = (2*(temp[i])**4/(np.pi)**2)*integrand_vals[i]*w_gauss[i]
        
    p_e = np.sum(vals)
    p = p_e+p_y+p_v+p_vs
    
    da = np.sqrt(8*np.pi/(3*m_pl**2))*a/(p)**(-1/2)
    
    results_1 = []
    for e,dec,col in zip(e_array,decay,coll):
        result_1 = np.trapz(e**3*(dec+col),e)
        results_1.append(result_1)
    
    dQ = (m_s*n_s*a**3/life)-(temp_cm**4*a**3/(2*np.pi**2))*da*results_1 #in units of MeV^5
    
    dQ_units = dQ/t #converts to MeV^4/sec

    np.savez(saved_file, T = temp, T_cm = temp_cm, dQdt = dQ, pn_rate = p_n, np_rate = n_p, rhonu = p_v)

    return dQ,p_v
    
    

In [4]:
imp_values("mass-300-life-0.030-new.npz","testing_function")

(array([ 1.72665600e-19,  1.72861361e-19,  1.72406042e-19,  1.71871359e-19,
         1.71332379e-19,  1.70753469e-19,  1.70154606e-19,  1.69529665e-19,
         1.68875401e-19,  1.68219667e-19,  1.67483831e-19,  1.66738637e-19,
         1.65958411e-19,  1.65114725e-19,  1.64302046e-19,  1.63381911e-19,
         1.62432913e-19,  1.61436730e-19,  1.60408309e-19,  1.59277286e-19,
         1.58104831e-19,  1.56819463e-19,  1.55553841e-19,  1.54154956e-19,
         1.52620925e-19,  1.51093125e-19,  1.49401748e-19,  1.47622295e-19,
         1.45661684e-19,  1.43619550e-19,  1.41327938e-19,  1.38944477e-19,
         1.36256711e-19,  1.33482552e-19,  1.30247449e-19,  1.26973269e-19,
         1.23518641e-19,  1.19964436e-19,  1.15846290e-19,  1.11362501e-19,
         1.06822193e-19,  1.04096421e-19,  1.01133258e-19,  9.69333845e-20,
         9.29400101e-20,  8.88548494e-20,  8.52157274e-20,  8.22586898e-20,
         7.85191077e-20,  7.44219409e-20,  7.04333896e-20,  6.69774849e-20,
         6.3